In [10]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
import os

# カレントディレクトリを取得
current_dir = os.getcwd()

# ファイルの相対パスを設定
data_file_path = os.path.join(current_dir, 'FrcSub', 'data.txt')
q_matrix_file_path = os.path.join(current_dir, 'FrcSub', 'q.txt')

# ファイルの存在確認
if not os.path.exists(data_file_path):
    raise FileNotFoundError(f"{data_file_path} not found.")
if not os.path.exists(q_matrix_file_path):
    raise FileNotFoundError(f"{q_matrix_file_path} not found.")

# ファイルの読み込み
response_data = np.loadtxt(data_file_path, dtype=int)
q_matrix = np.loadtxt(q_matrix_file_path, dtype=int)

# DINAモデルの計算関数
# ここにDINAモデルの計算関数を追加
def dina_model_likelihood(attribute_profile, response_data, q_matrix, slip, guess):
    n_students, n_items = response_data.shape
    n_attributes = attribute_profile.shape[1]
    
    mastery = np.dot(attribute_profile, q_matrix.T) == q_matrix.sum(axis=1)
    prob_correct = mastery * (1 - slip) + (1 - mastery) * guess
    likelihood = prob_correct ** response_data * (1 - prob_correct) ** (1 - response_data)
    return np.prod(likelihood, axis=1)

def estimate_parameters(response_data, q_matrix):
    n_students, n_items = response_data.shape
    n_attributes = q_matrix.shape[1]

    # 全ての可能なアトリビュートプロファイルを生成
    attribute_profiles = np.array(np.meshgrid(*[[0, 1]] * n_attributes)).T.reshape(-1, n_attributes)

    # 初期値
    slip = 0.2
    guess = 0.2

    # 最尤推定
    def negative_log_likelihood(params):
        slip, guess = params
        likelihoods = np.zeros((n_students, attribute_profiles.shape[0]))
        for i, profile in enumerate(attribute_profiles):
            likelihoods[:, i] = dina_model_likelihood(profile.reshape(1, -1), response_data, q_matrix, slip, guess)
        student_likelihoods = np.sum(likelihoods, axis=1)
        return -np.sum(np.log(student_likelihoods))

    result = minimize(negative_log_likelihood, [slip, guess], bounds=[(0, 1), (0, 1)])
    slip, guess = result.x

    # 学生ごとの習得アトリビュートを推定
    estimated_attributes = []
    for responses in response_data:
        likelihoods = [dina_model_likelihood(profile.reshape(1, -1), responses.reshape(1, -1), q_matrix, slip, guess) 
                       for profile in attribute_profiles]
        estimated_attributes.append(attribute_profiles[np.argmax(likelihoods)])

    return np.array(estimated_attributes), slip, guess

# モデル推定
estimated_attributes, slip, guess = estimate_parameters(response_data, q_matrix)

# 結果を表示
print("推定された習得アトリビュート:")
print(estimated_attributes)
print(f"推定されたスリップ確率: {slip}")
print(f"推定された推測確率: {guess}")


/var/folders/zg/773ptkr55z99zw26dvy19_v00000gn/T/ipykernel_92518/3511041662.py:52: RuntimeWarning: divide by zero encountered in log
  return -np.sum(np.log(student_likelihoods))


推定された習得アトリビュート:
[[1 1 1 ... 1 1 1]
 [1 1 1 ... 0 1 1]
 [1 1 0 ... 0 1 0]
 ...
 [0 0 0 ... 0 0 0]
 [1 1 0 ... 1 1 1]
 [0 1 0 ... 1 1 1]]
推定されたスリップ確率: 0.2
推定された推測確率: 0.2


In [11]:
from collections import Counter
import pandas as pd

# アトリビュートプロファイルごとの人数を計算
attribute_counts = Counter([tuple(attr) for attr in estimated_attributes])

# 結果をデータフレームに変換して見やすく表示
attribute_counts_df = pd.DataFrame(
    list(attribute_counts.items()), 
    columns=["Attribute Profile", "Count"]
)

# カウント順にソート
attribute_counts_df = attribute_counts_df.sort_values(by="Count", ascending=False)

# 結果を表示
print(attribute_counts_df)

           Attribute Profile  Count
12  (1, 1, 1, 1, 1, 1, 1, 1)    120
7   (0, 0, 0, 0, 0, 0, 0, 0)     60
11  (0, 1, 0, 0, 0, 0, 1, 1)     55
6   (0, 1, 0, 0, 0, 0, 0, 0)     38
1   (1, 1, 1, 1, 1, 0, 1, 1)     33
14  (0, 1, 0, 1, 0, 1, 1, 1)     24
3   (0, 1, 1, 1, 1, 1, 1, 1)     20
4   (0, 0, 0, 0, 0, 0, 1, 1)     17
5   (0, 1, 0, 0, 0, 0, 1, 0)     17
8   (1, 1, 0, 1, 0, 1, 1, 1)     15
9   (0, 1, 0, 1, 1, 1, 1, 1)     15
19  (0, 1, 0, 1, 0, 1, 1, 0)     10
37  (0, 1, 0, 1, 0, 0, 1, 1)      8
24  (0, 0, 0, 1, 0, 1, 1, 1)      8
17  (1, 1, 0, 1, 0, 1, 1, 0)      7
0   (1, 1, 1, 0, 1, 1, 1, 1)      7
10  (0, 0, 0, 0, 0, 0, 1, 0)      7
28  (0, 0, 0, 1, 0, 1, 1, 0)      6
15  (1, 1, 1, 0, 1, 1, 1, 0)      5
13  (0, 1, 0, 0, 1, 1, 1, 0)      5
21  (1, 1, 0, 1, 1, 1, 1, 1)      5
34  (0, 1, 1, 1, 1, 0, 1, 1)      4
23  (1, 1, 1, 1, 1, 1, 1, 0)      4
40  (0, 1, 0, 1, 1, 1, 1, 0)      4
25  (1, 0, 0, 1, 0, 1, 1, 1)      4
32  (0, 1, 0, 1, 1, 0, 1, 1)      4
29  (1, 1, 0, 0, 0, 0, 1, 1)